In [1]:
#import statements
import pandas as pd

#sklearn imports
from sklearn import preprocessing


%matplotlib inline 

#sets max columns so I can everything
pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth', -1)

In [2]:
#load data

dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")

In [3]:
#specify if data is train/test
dfTrain['DataType'] = 'Train'
dfTest['DataType'] = 'Test'

In [4]:
#combine data frames
dfMaster = pd.concat([dfTrain,dfTest])

In [5]:
#check length of master columns
len(dfMaster.columns)

82

In [6]:
#creation of variable explanation cheat sheet
dfDesciptOnly = pd.read_csv('data_descriptions_only.csv',header =0)
dfDesciptOnly['Variable'],dfDesciptOnly['Description'] = dfDesciptOnly['Description'].str.split(':').str

In [7]:
#create df of variables only and name it "Variable"
ColumnNames = pd.DataFrame(dfMaster.columns)
ColumnNames.columns = ['Variable']

In [8]:
dfCheatSheet = pd.merge(ColumnNames,dfDesciptOnly, on = 'Variable')

In [9]:
#create filter to differentiate from Train and Test data
TrainFilter = dfMaster['DataType']=='Train'

In [10]:
#Data Cleaning

#identify & drop columns where 95% of column data is missing
#if missing too much data, imputing is unlikely to be helpful

#5% missing threshold
threshold5per = len(dfMaster[TrainFilter])*.05
#print(threshold95per)

#isolate columns where more than 5% of data is missing 
#can index the df.columns based on boolean value
columnsToDrop = dfMaster[TrainFilter].columns[dfMaster[TrainFilter].isnull().sum()>threshold5per]

#drop columns where more than 5% of data is missing 
dfMaster.drop(labels = columnsToDrop, inplace=True, axis =1 )

In [11]:
len(dfMaster.columns)

71

In [12]:
#Remove columns where no seperation between value
#if only have only have one or two, unlikely to provide us any extra info

In [13]:
for col in dfMaster.columns:
    if (len(dfMaster[TrainFilter][col].unique())<=4):
        print(dfMaster[TrainFilter][col].value_counts())
        print()

#for each column in df master
#if the unique values in the column is less than equal to for
#print the the value counts of the column

0.0    856
1.0    588
2.0    15 
3.0    1  
Name: BsmtFullBath, dtype: int64

0.0    1378
1.0    80  
2.0    2   
Name: BsmtHalfBath, dtype: int64

Y    1365
N    95  
Name: CentralAir, dtype: int64

Train    1460
Name: DataType, dtype: int64

TA    906
Gd    488
Ex    52 
Fa    14 
Name: ExterQual, dtype: int64

0    690
1    650
2    115
3    5  
Name: Fireplaces, dtype: int64

2    768
1    650
3    33 
0    9  
Name: FullBath, dtype: int64

0    913
1    535
2    12 
Name: HalfBath, dtype: int64

1    1392
2    65  
3    2   
0    1   
Name: KitchenAbvGr, dtype: int64

TA    735
Gd    586
Ex    100
Fa    39 
Name: KitchenQual, dtype: int64

Lvl    1311
Bnk    63  
HLS    50  
Low    36  
Name: LandContour, dtype: int64

Gtl    1382
Mod    65  
Sev    13  
Name: LandSlope, dtype: int64

Reg    925
IR1    484
IR2    41 
IR3    10 
Name: LotShape, dtype: int64

Y    1340
N    90  
P    30  
Name: PavedDrive, dtype: int64

Pave    1454
Grvl    6   
Name: Street, dtype: int64

AllPub   

In [14]:
#Based on the above analysis, drop Street and Utilities.  Neither category has enough enough for meaningful info
dfMaster.drop(labels = ['Street','Utilities'], inplace=True, axis =1 )

In [15]:
len(dfMaster.columns)

69

In [16]:
dfMaster.to_csv('dfMasterCleaned.csv')

In [17]:
#reload cleaned data

dfMasterClean = pd.read_csv('dfMasterCleaned.csv',index_col = 0)

In [18]:
dfMasterClean.dtypes.value_counts()

object     33
int64      26
float64    10
dtype: int64

In [19]:
objectFilter = list(dfMasterClean.dtypes == 'object')
intFilter = list(dfMaster.dtypes != 'object')

In [20]:
print(dfMasterClean.columns[objectFilter])
print(dfMasterClean.columns[intFilter])

Index(['BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'DataType',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Foundation', 'Functional', 'Heating', 'HeatingQC', 'HouseStyle',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape',
       'MSZoning', 'MasVnrType', 'Neighborhood', 'PavedDrive', 'RoofMatl',
       'RoofStyle', 'SaleCondition', 'SaleType'],
      dtype='object')
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea',
       'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold',
       'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',
       'ScreenPorch', 'TotRmsAbvGrd', 'Total

In [21]:
#create list for indexing int and object variables
objectFilter = ['BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'DataType',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Foundation', 'Functional', 'Heating', 'HeatingQC', 'HouseStyle',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape',
       'MSZoning', 'MasVnrType', 'Neighborhood', 'PavedDrive', 'RoofMatl',
       'RoofStyle', 'SaleCondition', 'SaleType']

intFilter = ['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea',
       'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold',
       'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',
       'ScreenPorch', 'TotRmsAbvGrd', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd', 'YrSold']

In [22]:
dfMasterClean[intFilter].head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GrLivArea,HalfBath,Id,KitchenAbvGr,LotArea,LowQualFinSF,MSSubClass,MasVnrArea,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,0,2,548.0,2.0,1710,1,1,1,8450,0,60,196.0,0,2,61,5,7,0,208500.0,0,8,856.0,0,2003,2003,2008
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,1,2,460.0,2.0,1262,0,2,1,9600,0,20,0.0,0,5,0,8,6,0,181500.0,0,6,1262.0,298,1976,1976,2007
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,1,2,608.0,2.0,1786,1,3,1,11250,0,60,162.0,0,9,42,5,7,0,223500.0,0,6,920.0,0,2001,2002,2008
3,961,756,0,3,216.0,0.0,1.0,0.0,540.0,272,1,1,642.0,3.0,1717,0,4,1,9550,0,70,0.0,0,2,35,5,7,0,140000.0,0,7,756.0,0,1915,1970,2006
4,1145,1053,0,4,655.0,0.0,1.0,0.0,490.0,0,1,2,836.0,3.0,2198,1,5,1,14260,0,60,350.0,0,12,84,5,8,0,250000.0,0,9,1145.0,192,2000,2000,2008


In [23]:
#identify int variables with missing values
missingIntVar = list(dfMasterClean[intFilter].columns[dfMasterClean[intFilter].isnull().sum()>0])

#remove SalePrice because need it to train on
missingIntVar = (missingIntVar[:7] +[missingIntVar[9]])

In [24]:
missingIntVar

['BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtUnfSF',
 'GarageArea',
 'GarageCars',
 'TotalBsmtSF']

In [25]:
#count the number of missing values before so see if anything is actually done
dfMasterClean[missingIntVar].isnull().sum()

BsmtFinSF1      1
BsmtFinSF2      1
BsmtFullBath    2
BsmtHalfBath    2
BsmtUnfSF       1
GarageArea      1
GarageCars      1
TotalBsmtSF     1
dtype: int64

In [26]:
#spot check the one null vaue to see if it will get fixed
print(dfMasterClean['BsmtFinSF1'][dfMasterClean['BsmtFinSF1'].isnull()])
print(dfMasterClean['GarageCars'][dfMasterClean['GarageCars'].isnull()])

660   NaN
Name: BsmtFinSF1, dtype: float64
1116   NaN
Name: GarageCars, dtype: float64


In [27]:
#However, mean won't work for all these missing points.  Mode would be a better fit for discrete numbers 
#i.e 2 car garage instead of a 1.8 car garage 
"""
Mean
BsmtFinSF1
BsmtFinSF2 
BsmtUnfSF
GarageArea
TotalBsmtSF

Mode
BsmtFullBath    
BsmtHalfBath
GarageCars
"""

missingMean = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','GarageArea','TotalBsmtSF']
missingMode = ['BsmtFullBath','BsmtHalfBath','GarageCars']

In [28]:
#cleaning missing mean values
dfMasterClean[missingMean] = dfMasterClean[missingMean].fillna(dfMasterClean[missingMean].mean())

In [29]:
dfMasterClean[missingMean].isnull().sum()

BsmtFinSF1     0
BsmtFinSF2     0
BsmtUnfSF      0
GarageArea     0
TotalBsmtSF    0
dtype: int64

In [30]:
dfMasterClean['BsmtFinSF1'].iloc[660]
#says that probably doesn't have a basement which probably is correct

0.0

In [31]:
#spot check one mode null value to see if it will get fixed

print(dfMasterClean['BsmtFullBath'][dfMasterClean['BsmtFullBath'].isnull()])
print(dfMasterClean['BsmtHalfBath'][dfMasterClean['BsmtHalfBath'].isnull()])
print(dfMasterClean['GarageCars'][dfMasterClean['GarageCars'].isnull()])

660   NaN
728   NaN
Name: BsmtFullBath, dtype: float64
660   NaN
728   NaN
Name: BsmtHalfBath, dtype: float64
1116   NaN
Name: GarageCars, dtype: float64


In [32]:
dfMasterClean[missingMode].isnull().sum()

BsmtFullBath    2
BsmtHalfBath    2
GarageCars      1
dtype: int64

In [33]:
#clean mode values
dfMasterClean[missingMode]=dfMasterClean[missingMode].fillna(dfMasterClean[missingMode].mode().iloc[0])

#mode is a series, so need to start at the begininning to get it to work
#df[cols]=df[cols].fillna(df.mode().iloc[0])

In [34]:
dfMasterClean[missingMode].isnull().sum()

BsmtFullBath    0
BsmtHalfBath    0
GarageCars      0
dtype: int64

In [35]:
#Confirm that int values are fixed
dfMasterClean[missingIntVar].isnull().sum()

BsmtFinSF1      0
BsmtFinSF2      0
BsmtFullBath    0
BsmtHalfBath    0
BsmtUnfSF       0
GarageArea      0
GarageCars      0
TotalBsmtSF     0
dtype: int64

In [36]:
#cleaning for missing object filters
#missingIntVar = list(dfMasterClean[intFilter].columns[dfMasterClean[intFilter].isnull().sum()>0])
missingObjVar = list(dfMasterClean[objectFilter].columns[dfMasterClean[objectFilter].isnull().sum()>0])

In [37]:
dfMasterClean[missingObjVar].isnull().sum()

BsmtCond        82
BsmtExposure    82
BsmtFinType1    79
BsmtFinType2    80
BsmtQual        81
Electrical      1 
Exterior1st     1 
Exterior2nd     1 
Functional      2 
KitchenQual     1 
MSZoning        4 
MasVnrType      24
SaleType        1 
dtype: int64

In [38]:
#Chance that I don't need to actually replace some N/A because means it doesn't apply

In [39]:
#Categorical labeling 

'''
#N/A means doesn't apply
BsmtCond 
BsmtExposure
BsmtFinType1
BsmtFinType2 
BsmtQual

#Need to replace N/A
Electrical
Exterior1st      
Exterior2nd 
Functional   
KitchenQual
MSZoning
MasVnrType
SaleType
'''

ObjToConvert = ['BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtQual']
ObjMissingMode = ['Electrical','Exterior1st','Exterior2nd','Functional','KitchenQual','MSZoning','MasVnrType','SaleType']

In [40]:
dfMasterClean[ObjMissingMode].isnull().sum()

Electrical     1 
Exterior1st    1 
Exterior2nd    1 
Functional     2 
KitchenQual    1 
MSZoning       4 
MasVnrType     24
SaleType       1 
dtype: int64

In [41]:
#clean missing mode numbers
dfMasterClean[ObjMissingMode]=dfMasterClean[ObjMissingMode].fillna(dfMasterClean[ObjMissingMode].mode().iloc[0])

In [42]:
dfMasterClean[ObjMissingMode].isnull().sum()

Electrical     0
Exterior1st    0
Exterior2nd    0
Functional     0
KitchenQual    0
MSZoning       0
MasVnrType     0
SaleType       0
dtype: int64

In [43]:
#Filled in missing values.  Now just need to convert categorical objects into integers.  

In [44]:
#converting missing values
dfMasterClean[objectFilter].info() #check to see how it is before

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 33 columns):
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtQual         2838 non-null object
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
DataType         2919 non-null object
Electrical       2919 non-null object
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-null object
Foundation       2919 non-null object
Functional       2919 non-null object
Heating          2919 non-null object
HeatingQC        2919 non-null object
HouseStyle       2919 non-null object
KitchenQual      2919 non-null object
LandContour      2919 non-null object
LandSlope        2919 non-null object
LotConfig        29

In [45]:
#Convert objects into ints

#load Label Encoder
le = preprocessing.LabelEncoder()

In [46]:
#converting all objects to strings (sometimes label encoder reads objects as floats instead of string)
dfMasterClean[objectFilter] = dfMasterClean[objectFilter].astype(str)

In [47]:
dfMasterClean[objectFilter].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 33 columns):
BldgType         2919 non-null object
BsmtCond         2919 non-null object
BsmtExposure     2919 non-null object
BsmtFinType1     2919 non-null object
BsmtFinType2     2919 non-null object
BsmtQual         2919 non-null object
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
DataType         2919 non-null object
Electrical       2919 non-null object
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-null object
Foundation       2919 non-null object
Functional       2919 non-null object
Heating          2919 non-null object
HeatingQC        2919 non-null object
HouseStyle       2919 non-null object
KitchenQual      2919 non-null object
LandContour      2919 non-null object
LandSlope        2919 non-null object
LotConfig        29

In [48]:
#applies the label encoder transform to all object variables

dfMasterClean[objectFilter] = dfMasterClean[objectFilter].apply(le.fit_transform)

In [49]:
dfMasterClean.head()
#should have no objects and I don't

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,DataType,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,0,3,3,706.0,0.0,2,5,1.0,0.0,2,150.0,1,2,2,1,4,0,4,2,12,13,0,2,2,6,548.0,2.0,1710,1,1,0,5,1,1,2,3,0,8450,4,3,0,60,3,196.0,1,0,2,5,61,5,7,2,0,1,1,4,208500.0,8,0,8,856.0,0,2003,2003,2008
1,1262,0,0,3,0,3,1,978.0,0.0,0,5,0.0,1.0,2,284.0,1,1,2,1,4,0,4,3,8,8,1,1,2,6,460.0,2.0,1262,0,1,0,2,2,1,3,3,0,9600,2,3,0,20,3,0.0,2,0,5,24,0,8,6,2,0,1,1,4,181500.0,8,0,6,1262.0,298,1976,1976,2007
2,920,866,0,3,0,3,2,486.0,0.0,2,5,1.0,0.0,2,434.0,1,2,2,1,4,0,4,2,12,13,1,2,2,6,608.0,2.0,1786,1,1,0,5,3,1,2,3,0,11250,4,0,0,60,3,162.0,1,0,9,5,42,5,7,2,0,1,1,4,223500.0,8,0,6,920.0,0,2001,2002,2008
3,961,756,0,3,0,1,3,216.0,0.0,0,5,1.0,0.0,3,540.0,1,2,2,1,4,272,4,3,13,15,1,0,1,6,642.0,3.0,1717,0,1,2,5,4,1,2,3,0,9550,0,0,0,70,3,0.0,2,0,2,6,35,5,7,2,0,1,1,0,140000.0,8,0,7,756.0,0,1915,1970,2006
4,1145,1053,0,4,0,3,0,655.0,0.0,2,5,1.0,0.0,2,490.0,1,2,2,1,4,0,4,2,12,13,1,2,2,6,836.0,3.0,2198,1,1,0,5,5,1,2,3,0,14260,2,0,0,60,3,350.0,1,0,12,15,84,5,8,2,0,1,1,4,250000.0,8,0,9,1145.0,192,2000,2000,2008


In [50]:
dfMasterClean.info()
#confirming no object variables

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 69 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
BedroomAbvGr     2919 non-null int64
BldgType         2919 non-null int64
BsmtCond         2919 non-null int64
BsmtExposure     2919 non-null int64
BsmtFinSF1       2919 non-null float64
BsmtFinSF2       2919 non-null float64
BsmtFinType1     2919 non-null int64
BsmtFinType2     2919 non-null int64
BsmtFullBath     2919 non-null float64
BsmtHalfBath     2919 non-null float64
BsmtQual         2919 non-null int64
BsmtUnfSF        2919 non-null float64
CentralAir       2919 non-null int64
Condition1       2919 non-null int64
Condition2       2919 non-null int64
DataType         2919 non-null int64
Electrical       2919 non-null int64
EnclosedPorch    2919 non-null int64
ExterCond        2919 non-null int64
ExterQual        2919 non-null int64
Exterior1st      2919 non-null i

In [51]:
#save to CVS
dfMasterClean.to_csv('FinalCleanedData.csv')

Cleaning Data Steps + Thoughts

* add train/test column to train/test data respectively and join them together.  Anything done to one is done to both
* drop columns where too much of the data is missing because will be unable to impute missing stuff
    * For this excerise I dropped if more than 5% of data was missing. But is there a good threshold?
    * 1% or 10%?  
* drop columns where not unique values are tiny and most of data falls in one category
* use fillna to fill in missing values
    * for int variables, need to figure out the benefits of using mean, median, or mode (which was helpful for when number should be discrete)
    * for categorical variables, use mode
* use label encoder to convert categorical variables into numerical ones
    * to what extent does the number conversion matter?  Does the number/order of the numbers affect most ML algorithms? 
    * doesn't make sense to me to create tons and tons of dummy variables for 33 different variables